In [249]:
import pandas as pd

data = pd.read_csv("CP_count_PUMA.csv")
data

,smiles,1_1,10_19,10_20,100_277,100_278,104_291,105_355,105_369,105_361,...,Cells_Neighbors_FirstClosestObjectNumber_Adjacent,Cells_Neighbors_SecondClosestObjectNumber_5,Cells_Neighbors_SecondClosestObjectNumber_Adjacent,Cells_Parent_Nuclei,Cytoplasm_Number_Object_Number,Cytoplasm_Parent_Cells,Cytoplasm_Parent_Nuclei,Nuclei_Neighbors_FirstClosestObjectNumber_1,Nuclei_Neighbors_SecondClosestObjectNumber_1,Nuclei_Number_Object_Number
0,CCOC(=O)c1ccc(NC(=S)N2CCSC2c2ccc(OC)cc2)cc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.765625,0.890625,0.890625,1.328125,1.328125,1.328125,1.328125,0.960938,1.320312,1.328125
1,O=C(CC1NC(=O)NC1=O)Nc1cccc2ccccc12,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.078125,2.718750,2.718750,2.503906,2.503906,2.503906,2.503906,3.242188,2.578125,2.503906
2,CC(Nc1nc(nc2ccccc12)N1CCCC1)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.765625,-8.140625,-8.140625,-8.250000,-8.250000,-8.250000,-8.250000,-7.710938,-7.546875,-8.250000
3,CCCC(Oc1ccc(Br)cc1)c1nc2c3cc(OC)c(OC)cc3nc(S)n2n1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.320312,-7.828125,-7.828125,-7.796875,-7.804688,-7.796875,-7.796875,-7.859375,-7.695312,-7.804688
4,CC1=C(C(NC(=O)N1)c1ccc(F)cc1)C(=O)OCc1ccc2OCOc2c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-10.953125,-10.390625,-10.390625,-10.960938,-10.960938,-10.960938,-10.960938,-11.406250,-10.781250,-10.960938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16165,COC(=O)[C@H]1[C@H](CO)[C@H]2Cn3c(=O)c(\C=C\C)c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.039062,-2.031250,-2.031250,-1.550781,-1.550781,-1.550781,-1.550781,-1.546875,-2.945312,-1.550781
16166,C\C=C\c1ccc2n(C[C@H]3[C@H](CO)[C@H](N(C)[C@@H]...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.851562,-0.359375,-0.359375,-0.835938,-0.835938,-0.835938,-0.835938,-1.328125,-0.804688,-0.835938
16167,C\C=C\c1ccc2n(C[C@@H]3[C@@H](CO)[C@@H](N(C)[C@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.851562,-3.359375,-3.359375,-3.835938,-3.835938,-3.835938,-3.835938,-3.328125,-4.054688,-3.835938
16168,C\C=C\c1ccc2n(C[C@H]3[C@H](CO)[C@@H](C(=O)N[C@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.039062,-2.460938,-2.460938,-3.300781,-3.300781,-3.300781,-3.300781,-3.382812,-2.937500,-3.300781


In [250]:
assays_list = data.columns.to_list()[1:-13]
len(assays_list)

270

In [251]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
from tqdm import tqdm


In [252]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier


def load_and_preprocess_data(fold):
    train_path = f"PUMA/predictions/chemical_cv{fold}/assay_matrix_discrete_train_scaff.csv"
    test_path = f"PUMA/predictions/chemical_cv{fold}/assay_matrix_discrete_test_scaff.csv"

    train_data = pd.read_csv(train_path)[["smiles"]]
    train_data = pd.merge(train_data, data, on="smiles")
    
    test_data = pd.read_csv(test_path)[["smiles"]]
    test_data = pd.merge(test_data, data, on="smiles")
    
    return train_data, test_data



In [274]:
import numpy as np
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.utils import class_weight
from sklearn.model_selection import GridSearchCV

# Initialize a list to store AUC scores for all folds
all_auc_scores = []

# Initialize an empty list to hold models for each fold
models_per_fold = []

# Define the fold indices (adjust based on your data split)
folds = ["0", "1", "2", "3", "4"]

best_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)

# Loop through each fold, leaving it out as the test set and training on the others
for fold in tqdm(folds):
    
    if(best_clf):      
        del(best_clf)
        
    train_data, _ = load_and_preprocess_data(fold)
        
    # Initialize lists to store data for the current fold's training set
    X_train_combined = pd.DataFrame()
    y_train_combined = pd.Series()
    
    # Loop through all assays and collect data
    for assay in assays_list:
        train_data_assay = train_data[~train_data[assay].isna()][[assay, "Cells_Number_Object_Number"]]
            
        # Append to combined training data
        X_train_combined = pd.concat([X_train_combined, train_data_assay[["Cells_Number_Object_Number"]]], axis=0)
        y_train_combined = pd.concat([y_train_combined, train_data_assay[assay].astype(int)], axis=0)
            
    print(len(X_train_combined))
    print(len(y_train_combined))

    best_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)
    best_clf.fit(X_train_combined, y_train_combined)
       
    # Store the trained model
    models_per_fold.append(best_clf)

# Predict for each individual fold using the corresponding model
for idx, fold in tqdm(enumerate(folds)):
    
    # Load and preprocess data for the current test fold
    _, test_data = load_and_preprocess_data(fold)
    
    # Get the trained model for this fold (trained on other folds)
    clf = models_per_fold[idx]
    
    # Loop through each assay in the test set for the current fold
    for assay in tqdm(assays_list):
        test_data_assay = test_data[~test_data[assay].isna()][[assay, "Cells_Number_Object_Number"]]
        
        # Separate features and target variable
        X_test = test_data_assay[["Cells_Number_Object_Number"]]
        y_test = test_data_assay[assay].astype(int)

        if len(y_test) == 0:
            print(f"Skipping assay {assay} in fold {fold} due to no test data")
            continue
        
        # Make predictions using the model for the current fold
        y_pred = clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)[:, 1]

        # Calculate AUC score
        auc_score = np.nan    
        try:
            auc_score = roc_auc_score(y_test, y_pred_proba)
        except:
            continue

        # Store the AUC score along with fold and assay information
        all_auc_scores.append({"Fold": fold, "Task": assay, "AUC": auc_score})

# Convert the AUC results to a DataFrame for convenience
auc_df = pd.DataFrame(all_auc_scores)

# Save the DataFrame to a CSV file
auc_df.to_csv("auc_scores_across_folds.csv", index=False)

# Display the AUC DataFrame
auc_df


  0%|          | 0/5 [00:00<?, ?it/s]/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


466484
466484


 20%|██        | 1/5 [00:16<01:06, 16.73s/it]

469311
469311


 40%|████      | 2/5 [00:33<00:50, 16.98s/it]

...

465957
465957


 80%|████████  | 4/5 [01:10<00:18, 18.03s/it]

470810
470810


100%|██████████| 5/5 [01:27<00:00, 17.56s/it]
0it [00:00, ?it/s]
100%|██████████| 270/270 [00:06<00:00, 43.24it/s]
1it [00:06,  6.58s/it]
100%|██████████| 270/270 [00:06<00:00, 43.44it/s]
2it [00:13,  6.57s/it]
100%|██████████| 270/270 [00:06<00:00, 43.48it/s]
3it [00:19,  6.56s/it]
100%|██████████| 270/270 [00:06<00:00, 43.37it/s]
4it [00:26,  6.56s/it]
100%|██████████| 270/270 [00:06<00:00, 43.90it/s]
5it [00:32,  6.55s/it]


,Fold,Task,AUC
0,0,1_1,0.476190
1,0,100_277,0.625000
2,0,100_278,0.500000
3,0,104_291,0.820513
4,0,105_369,0.566667
...,...,...,...
1006,4,9_18,0.500000
1007,4,90_256,0.178571
1008,4,91_257,1.000000
1009,4,92_258,1.000000


In [275]:
mean_auc = auc_df.groupby("Task").mean().reset_index(drop=False)
mean_auc

,Task,AUC
0,100_277,0.753333
1,100_278,0.629894
2,104_289,0.750962
3,104_291,0.788462
4,104_295,0.714286
...,...,...
264,92_258,0.594643
265,93_259,0.519436
266,9_14,0.500000
267,9_17,1.000000


In [276]:
len(mean_auc[mean_auc["AUC"]==1])

4

In [277]:
len(mean_auc[mean_auc["AUC"]>0.90])

12

In [278]:
len(mean_auc[mean_auc["AUC"]>0.80])

31

In [279]:
mean_auc.to_csv("scaffold_mean_AUCSeal.csv")

In [280]:
mean_auc.mean()

Task         inf
AUC     0.588322
dtype: float64

In [281]:
mean_auc.std()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


AUC    0.180173
dtype: float64

In [282]:
median_auc = auc_df.groupby("Task").median().reset_index(drop=False)
median_auc.to_csv("scaffold_median_AUCSeal.csv")

In [283]:
len(median_auc[median_auc["AUC"]==1])

6

In [284]:
len(median_auc[median_auc["AUC"]>0.90])

15